In [8]:
import numpy as np
import pandas as pd
import re

import warnings
warnings.filterwarnings("ignore")

the dataset contains THREE YEARS of recorded incidents ("alarms") of broken fridges with some specific infos:
* When the fridge breaks down this is reported in the computer system and the CMS company sends an ingeneer to fix it and they it asks the Supermarket to pay for the service (maybe its a good business in Colombia ;)
* the original file has 150MB
* 600.000 records from 2012-2015

The questions are:
* If the company has details (data, logs etc) on all the "alarms" in the year 2012 - 2015, it wants to predicts how many alarms in each store in each month will happen in 2016?
* why?
* which shops has largest smallest number of alarms (maybe normalize per size of the shop)
* what data would you suggest the manager to collect to make the dataser more complete
* report and ask questions


Lets look at the data
* Work Order Number ----------------------------------------- 12939001 (like a factura, work done was stored as a number)
* Site Number -------------------------------------------------- A0052 (each store that company maintains has a number)
* Site Type ---------------------------------------------- SUPERMARKET (type of store: Supermarket(big)/Convenience(small))
* Date Created -------------------------------------- 02/02/2012 09:45 (date and time that the alarm occured)
* Date Completed  ----------------------------------- 02/02/2012 21:10 (date and time that the alarm was silenced)
* Provider Name  ------------------------------- CMS Solutions Limited (name of the maintenance company)
* WO Current Status ---------------------------------------------- EOL (I dont remember)
* Asset Category --------------------------------------- REFRIGERATION (type of asset, they also fix aircond, light etc)
* Problem Type Name ---------- Fixture Critical Over Temperature Alarm (type of alarm)
* Requested By ---------------------------- Urgent Refrigeration Alarm (priority of the alarm)
* WO Priority ------------------------------------------- Standard SLA (for accountancy of the company)
* Manufacturer ---------------------------------------------- LINDE AG (manufacturer of the asset that made the alarm)
* Asset In-Service Date ----------------------------- 23/05/2012 05:00 (when the machine was first connected)
* Level 1 Organization Name ------------------------------- Region S01 (the company divied the country into regions)
* Site Zipcode ----------------------------------------------- CR0 4XT (postal code of the store - helps to locate it)
* Sales Sq. Ft. ---------------------------------------------- 48409.0 (the size of the stor in square feet)
* Asset Barcode ------------------------------------------ SSL00319496 (number of the assed (fridge), a label)
* Client Product Name -------------------------------------------- NaN (dont remember)
* Asset Type ---------------------------------------- REMOTE MULTIDECK (fridge can be open, closed, cabinet etc)
* Asset Status ---------------------------------------------- Inactive (is the asset active today -time of generating dataset)

In [9]:
data = pd.read_csv("data/CPT_PREDICTIVE_PPM.csv",encoding='windows-1252')
data.head()

,Work Order Number,Site Number,Site Type,Date Created,Date Completed,Provider Name,WO Current Status,Asset Category,Problem Type Name,Requested By,WO Priority,Manufacturer,Asset In-Service Date,Level 1 Organization Name,Site Zipcode,Sales Sq. Ft.,Asset Barcode,Client Product Name,Asset Type,Asset Status
0,12904684,A0892,SUPERMARKET,01/02/2012 03:05,01/02/2012 10:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,01/06/2011 15:19,Region N10,DN31 1UF,42639.0,SSL00567391,Frozen Food (-18C or less),HALF GLASS DOOR & WELL,In Service
1,12904685,A0892,SUPERMARKET,01/02/2012 03:05,01/02/2012 09:59,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,01/06/2011 15:19,Region N10,DN31 1UF,42639.0,SSL00567398,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
2,12904853,A0892,SUPERMARKET,01/02/2012 05:10,01/02/2012 10:01,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,01/06/2011 15:19,Region N10,DN31 1UF,42639.0,SSL00567399,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
3,12927899,A0416,SUPERMARKET,01/02/2012 08:40,02/02/2012 13:03,CMS Solutions Limited,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,LINDE,01/08/2002 05:00,Region S01,BR4 0PU,13423.0,SSL00225160,"Meat, Fish and Poultry (3C or less)",REMOTE MULTIDECK,In Service
4,12932231,A0003,SUPERMARKET,01/02/2012 13:33,08/02/2012 19:39,CMS Solutions Limited,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,15/03/2009 04:00,Region S02,DA11 8JH,37870.0,SSL00583503,NaN,FULL GLASS DOOR,Scrapped


### Organizar columnas para llamar posteriormente

In [12]:
col = []
for columns in data.columns:
    #temp = columns.replace(" ","_")
    #temp = columns.replace("-","_")
    temp = columns.replace(".","")
    temp = temp.lower()
    col.append(temp)
data.columns = col
data.head()

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
0,12904684,A0892,SUPERMARKET,01/02/2012 03:05,01/02/2012 10:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,01/06/2011 15:19,Region N10,DN31 1UF,42639.0,SSL00567391,Frozen Food (-18C or less),HALF GLASS DOOR & WELL,In Service
1,12904685,A0892,SUPERMARKET,01/02/2012 03:05,01/02/2012 09:59,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,01/06/2011 15:19,Region N10,DN31 1UF,42639.0,SSL00567398,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
2,12904853,A0892,SUPERMARKET,01/02/2012 05:10,01/02/2012 10:01,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,01/06/2011 15:19,Region N10,DN31 1UF,42639.0,SSL00567399,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
3,12927899,A0416,SUPERMARKET,01/02/2012 08:40,02/02/2012 13:03,CMS Solutions Limited,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,LINDE,01/08/2002 05:00,Region S01,BR4 0PU,13423.0,SSL00225160,"Meat, Fish and Poultry (3C or less)",REMOTE MULTIDECK,In Service
4,12932231,A0003,SUPERMARKET,01/02/2012 13:33,08/02/2012 19:39,CMS Solutions Limited,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,15/03/2009 04:00,Region S02,DA11 8JH,37870.0,SSL00583503,NaN,FULL GLASS DOOR,Scrapped


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609989 entries, 0 to 609988
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   work_order_number          609989 non-null  int64  
 1   site_number                609989 non-null  object 
 2   site_type                  609989 non-null  object 
 3   date_created               609989 non-null  object 
 4   date_completed             577602 non-null  object 
 5   provider_name              609989 non-null  object 
 6   wo_current_status          609989 non-null  object 
 7   asset_category             609989 non-null  object 
 8   problem_type_name          609989 non-null  object 
 9   requested_by               527644 non-null  object 
 10  wo_priority                609989 non-null  object 
 11  manufacturer               609989 non-null  object 
 12  asset_in_service_date      506751 non-null  object 
 13  level_1_organization_name  60

Se observan 7 columnas incompletas, se revisan primero estas:

### **date_created | date_completed | asset_in_service_date**

In [14]:
data.date_completed[0] # formato: Fecha más horas y minutos	YYYY-MM-DDThh:mmTZD

'01/02/2012 10:00'

In [15]:
print( "Numero de valores en 'date_completed' con nan:",data[data.date_completed.isnull()]["work_order_number"].count() ) # valores con NaN en date_completed
print( "Porcentaje de datos perdidos respecto al tamaño del Dataset: "+str(round((32387/609989)*100 ,2))+"%" )
print( "Formato de valores en la columna:",type(data.date_completed[0]) )

Numero de valores en 'date_completed' con nan: 32387
Porcentaje de datos perdidos respecto al tamaño del Dataset: 5.31%
Formato de valores en la columna: <class 'str'>


In [16]:
%%time
data["date_created"] = pd.DatetimeIndex(data.date_created)
data["date_completed"] = pd.DatetimeIndex(data.date_completed)
data["asset_in_service_date"] = pd.DatetimeIndex(data.asset_in_service_date)

CPU times: total: 1min 24s
Wall time: 2min 17s


In [17]:
data[data.date_completed.isnull()].head(3)

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
13,12940592,A0003,SUPERMARKET,2012-02-02 12:02:00,NaT,CMS Solutions Limited,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00583504,NaN,FULL GLASS DOOR,Inactive
15,12940828,A0003,SUPERMARKET,2012-02-02 12:26:00,NaT,CMS Solutions Limited,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,ND,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00836662 -duplicate-1,NaN,FULL GLASS DOOR,Scrapped
25,12953604,A2221,SUPERMARKET,2012-03-02 20:15:00,NaT,Space Engineering services LTD,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CRIOSBANC,2009-06-17 05:00:00,Region C01,GL2 5SA,NaN,SSL00624924,NaN,FISH COUNTER,In Service


Considerando que hay fechas que el mismo dia se activo y se silencio la alarma, entonces completamos los resultados faltantes de date_completed usando el metodo ffill de pandas el cual llena valores faltantes entre cada gap de nan por un valor valido.

In [18]:
data.date_completed.fillna(axis=0,method='ffill',inplace=True)

In [19]:
data[(data.work_order_number == 12940592) | (data.work_order_number == 12940828)]

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
13,12940592,A0003,SUPERMARKET,2012-02-02 12:02:00,2012-02-02 13:38:00,CMS Solutions Limited,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00583504,NaN,FULL GLASS DOOR,Inactive
15,12940828,A0003,SUPERMARKET,2012-02-02 12:26:00,2012-02-02 14:01:00,CMS Solutions Limited,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,ND,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00836662 -duplicate-1,NaN,FULL GLASS DOOR,Scrapped


Observamos que para las ordenes "12940592" y "12940828" el valor que completa los NaT son validos en comparacion a datos que estan completos. Similarmente ocurre con la columna "asset_in_service_date"

In [20]:
data[data.asset_in_service_date.isnull()].head(3)

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
3964,13342929,A0852,SUPERMARKET,2012-09-03 12:15:00,2012-10-03 12:36:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,LINDE,NaT,Region N11,LE11 5AS,28039.0,SSL00633639,NaN,WELL,Scrapped
4070,13346368,A0893,SUPERMARKET,2012-09-03 18:55:00,2012-10-03 18:02:00,Montgomery Refrigeration Ltd,Invoice Processed,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RMS CONTROLS,NaT,Region N03,G74 4UN,38116.0,SSL00017276,NaN,COLDROOM CONTROL PANEL,In Service
4743,13359685,A0893,SUPERMARKET,2012-11-03 21:05:00,2012-11-04 14:27:00,Montgomery Refrigeration Ltd,Invoice Processed,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RMS CONTROLS,NaT,Region N03,G74 4UN,38116.0,SSL00017276,NaN,COLDROOM CONTROL PANEL,In Service


In [21]:
data.asset_in_service_date.fillna(axis=0,method='ffill',inplace=True)

In [22]:
data[(data.work_order_number == 13342929) | (data.work_order_number == 13346368) | (data.work_order_number == 13359685)]

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
3964,13342929,A0852,SUPERMARKET,2012-09-03 12:15:00,2012-10-03 12:36:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,LINDE,2011-02-14 05:00:00,Region N11,LE11 5AS,28039.0,SSL00633639,NaN,WELL,Scrapped
4070,13346368,A0893,SUPERMARKET,2012-09-03 18:55:00,2012-10-03 18:02:00,Montgomery Refrigeration Ltd,Invoice Processed,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RMS CONTROLS,2009-05-15 05:00:00,Region N03,G74 4UN,38116.0,SSL00017276,NaN,COLDROOM CONTROL PANEL,In Service
4743,13359685,A0893,SUPERMARKET,2012-11-03 21:05:00,2012-11-04 14:27:00,Montgomery Refrigeration Ltd,Invoice Processed,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RMS CONTROLS,2009-06-26 17:12:00,Region N03,G74 4UN,38116.0,SSL00017276,NaN,COLDROOM CONTROL PANEL,In Service


Efectivamente asset_in_service_date debe ser un resultado menor que date_created dado la descripcion del dataset, por lo que es una buena aproximacion este resultado para completar los datos faltantes.

In [23]:
print( "Numero de valores en 'date_created' con nan:",data[data.date_created.isnull()]["work_order_number"].count() )
print( "Numero de valores en 'date_completed' con nan:",data[data.date_completed.isnull()]["work_order_number"].count() )
print( "Numero de valores en 'asset_in_service_date' con nan:",data[data.asset_in_service_date.isnull()]["work_order_number"].count() )

Numero de valores en 'date_created' con nan: 0
Numero de valores en 'date_completed' con nan: 0
Numero de valores en 'asset_in_service_date' con nan: 0


### **requested_by**

In [24]:
type(data.requested_by[0]) # tipo de variable

str

In [25]:
print(data.requested_by.unique()[:5])
print(len(data.requested_by.unique()))

['Urgent Refrigeration Alarm'
 'URGENT Refrigeration Fixture Communication Alarm'
 'URGENT Refrigeration Fixture Monitoring Alarm'
 'Valve Open >50% for 3hrs sustained for 3 times within 24hrs (All Cabinets and Coldrooms) LIVE '
 'AirOn/Off Temp Diff >5°C for 8hr within 24hr period(ProvCab) LIVE ']
112


In [26]:
data[~data.requested_by.isnull()].tail()

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
527639,32704393,A0756,SUPERMARKET,2015-01-11 20:35:00,2015-01-11 20:08:00,Carter Refrigeration,Pend.,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,NUTTALL,2011-09-16 06:29:00,Region C02,ST16 2TF,41385.0,SSL00827276,"Provisions, Dairy and Prepared Produce (5C or ...",SERVE OVER,In Service
527640,32704394,A0885,SUPERMARKET,2015-01-11 20:35:00,2015-01-11 20:08:00,KB (Carters),Assigned,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,WILLIAMS REFRIGERATION,2010-10-28 00:00:00,Region N03,G53 7RH,35575.0,SSL00769586,Frozen Food (-18C or less),INTEGRAL FREEZER,In Service
527641,32704402,A4261,CONVENIENCE STORE,2015-01-11 20:38:00,2015-01-11 20:08:00,Space Engineering services LTD,Assigned,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,VERCO,2012-11-21 05:00:00,Region S09,PO4 0JT,NaN,SSL00495507,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
527642,32704447,A4248,CONVENIENCE STORE,2015-01-11 20:50:00,2015-01-11 20:08:00,CMS Solutions Limited,Assigned,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,EPTA,2012-11-05 05:00:00,Region S04-2,E8 4HR,NaN,SSL00818570,"Meat, Fish and Poultry (3C or less)",REMOTE MULTIDECK,In Service
527643,32704448,A0816,SUPERMARKET,2015-01-11 20:50:00,2015-01-11 20:08:00,Space Engineering services LTD,Pend.,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER RETAIL EQUIPMENT,2011-10-20 05:00:00,Region N11,NG16 2LY,24743.0,SSL00803138,Frozen Food (-18C or less),HALF GLASS DOOR END,In Service


In [27]:
data[data.requested_by.isnull()].head()

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
527644,12917191,D0470,DEPOT,2012-01-02 05:59:00,2012-03-02 10:47:00,CMS Self Delivery,EOL,REFRIGERATION,CMS S D - Ammonia Checks (Dartford) - Daily,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Depot,DA1 5PR,NaN,No Asset,NaN,NaN,NaN
527645,12920048,A2154,SUPERMARKET,2012-01-02 06:02:00,2012-03-13 13:14:00,Montgomery Refrigeration Ltd,EOL,REFRIGERATION,Montgomery - REMOTE AIR COOLED EXTERNAL CONDEN...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Region N02,EH11 2QW,49471.0,No Asset,NaN,NaN,NaN
527646,12920042,A2031,SUPERMARKET,2012-01-02 06:02:00,2012-05-03 14:26:00,Montgomery Refrigeration Ltd,EOL,REFRIGERATION,Montgomery - REMOTE AIR COOLED EXTERNAL CONDEN...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Region N03,G51 4BP,46224.0,No Asset,NaN,NaN,NaN
527647,12920044,A2017,SUPERMARKET,2012-01-02 06:02:00,2012-08-03 13:28:00,Montgomery Refrigeration Ltd,EOL,REFRIGERATION,Montgomery - REMOTE AIR COOLED EXTERNAL CONDEN...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Region N02,KY8 4LQ,13650.0,No Asset,NaN,NaN,NaN
527648,12920032,A4676,CONVENIENCE STORE,2012-01-02 06:02:00,2012-03-30 13:26:00,Montgomery Refrigeration Ltd,EOL,REFRIGERATION,Montgomery - REMOTE AIR COOLED EXTERNAL CONDEN...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Region N02-2,AB25 1EP,NaN,No Asset,NaN,NaN,NaN


Vemos que a partir del indice (527643) todos los valores de la columna "requested_by" son nulos

In [28]:
print( "Numero de valores en 'requested_by' con nan:",data[data.requested_by.isnull()]["work_order_number"].count() )

Numero de valores en 'requested_by' con nan: 82345


### **sales_sq_ft**

In [29]:
type(data.sales_sq_ft[0]) # tipo de variable

numpy.float64

In [30]:
data[data.sales_sq_ft.isnull()].head(3)

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
25,12953604,A2221,SUPERMARKET,2012-03-02 20:15:00,2012-04-02 09:00:00,Space Engineering services LTD,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CRIOSBANC,2009-06-17 05:00:00,Region C01,GL2 5SA,NaN,SSL00624924,NaN,FISH COUNTER,In Service
26,12954198,A2221,SUPERMARKET,2012-04-02 02:00:00,2012-07-02 21:21:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CRIOSBANC,2009-06-17 05:00:00,Region C01,GL2 5SA,NaN,SSL00624924,NaN,FISH COUNTER,In Service
58,13002141,A2167,SUPERMARKET,2012-07-02 15:41:00,2012-08-02 00:06:00,CMS Solutions Limited,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER RETAIL EQUIPMENT LTD,2011-01-01 05:00:00,Region S02,CT21 5BH,NaN,SSL00507794,"Provisions, Dairy and Prepared Produce (5C or ...",REMOTE MULTIDECK,In Service


In [31]:
data[(data.site_number == "A2221") & (data.sales_sq_ft.isnull())].head(3)

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
25,12953604,A2221,SUPERMARKET,2012-03-02 20:15:00,2012-04-02 09:00:00,Space Engineering services LTD,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CRIOSBANC,2009-06-17 05:00:00,Region C01,GL2 5SA,NaN,SSL00624924,NaN,FISH COUNTER,In Service
26,12954198,A2221,SUPERMARKET,2012-04-02 02:00:00,2012-07-02 21:21:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CRIOSBANC,2009-06-17 05:00:00,Region C01,GL2 5SA,NaN,SSL00624924,NaN,FISH COUNTER,In Service
277,13031014,A2221,SUPERMARKET,2012-11-02 01:55:00,2012-11-02 18:49:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER RETAIL EQUIPMENT,2009-06-17 05:00:00,Region C01,GL2 5SA,NaN,SSL00615845,Frozen Food (-18C or less),WELL,In Service


Indentificando el site_number (numero que representa la tienda) y buscando si tiene alojado algun nombre no logra identificar coincidencias, quiere decir que todos los resultados para esa tienda no contienen la columna "sales_sq_ft". como es un numero que representa el tamaño de la tienda entonces no se pueden completar los NaN con 0. debe ser un tamaño adecuado por lo que una aproximacion para las faltantes seria la media de la columna entera.

In [32]:
data.sales_sq_ft.fillna(value=data.sales_sq_ft.mean(),inplace=True)

In [33]:
print( "Numero de valores en 'sales_sq_ft' con nan:",data[data.sales_sq_ft.isnull()]["work_order_number"].count() )

Numero de valores en 'sales_sq_ft' con nan: 0


### **client_product_name**       

In [34]:
data[~data.client_product_name.isnull()].head(3)

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
0,12904684,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 10:00:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567391,Frozen Food (-18C or less),HALF GLASS DOOR & WELL,In Service
1,12904685,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 09:59:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567398,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
2,12904853,A0892,SUPERMARKET,2012-01-02 05:10:00,2012-01-02 10:01:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567399,Frozen Food (-18C or less),FULL GLASS DOOR,In Service


In [35]:
print(data.client_product_name.unique().tolist())
print("tamaño de categorias:",len(data.client_product_name.unique()))

['Frozen Food (-18C or less)', 'Meat, Fish and Poultry (3C or less)', nan, 'Provisions, Dairy and Prepared Produce (5C or less)', 'Soft Drinks/BWS (5C or less - alarms at 12C)', 'Produce (7C or less)', 'Pharmacy (2 - 8C)']
tamaño de categorias: 7


El nombre de la columna no tiene relacion con los valores que muestra, parece ser que es mas que todo una columna que muestra el tipo de uso que se le da a la nevera junto con su rango de enfriamiento. contiene 7 categorias (se puede separar una columna con las categorias y otra con el rango de enfriamiento)

In [36]:
type_of_use = []
cooling_range = []
for i in range(len(data)):
    try:
        type_temp = data.client_product_name[i].split("(")[0]
        max_cool_temp = re.findall("(-\d+|\d+)",data.client_product_name[i].split("(")[1])
        if len(max_cool_temp) > 1: cool_temp = f"({max_cool_temp[0]}-{max_cool_temp[1]})C - less"
        else: cool_temp = f"{max_cool_temp[0]}C - less"
        #print(cool_temp)
        type_of_use.append(type_temp)
        cooling_range.append(cool_temp)
    except:
        type_of_use.append(np.nan)
        cooling_range.append(np.nan)

data.insert(0,"preserves",type_of_use)
data.insert(0,"cooling_range",cooling_range)

In [37]:
data.head()

,cooling_range,preserves,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,...,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
0,-18C - less,Frozen Food,12904684,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 10:00:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567391,Frozen Food (-18C or less),HALF GLASS DOOR & WELL,In Service
1,-18C - less,Frozen Food,12904685,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 09:59:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567398,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
2,-18C - less,Frozen Food,12904853,A0892,SUPERMARKET,2012-01-02 05:10:00,2012-01-02 10:01:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567399,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
3,3C - less,"Meat, Fish and Poultry",12927899,A0416,SUPERMARKET,2012-01-02 08:40:00,2012-02-02 13:03:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,LINDE,2002-01-08 05:00:00,Region S01,BR4 0PU,13423.0,SSL00225160,"Meat, Fish and Poultry (3C or less)",REMOTE MULTIDECK,In Service
4,NaN,NaN,12932231,A0003,SUPERMARKET,2012-01-02 13:33:00,2012-08-02 19:39:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00583503,NaN,FULL GLASS DOOR,Scrapped


In [38]:
print(data.preserves.unique().tolist())
print("tamaño de categorias:",len(data.preserves.unique()))

['Frozen Food ', 'Meat, Fish and Poultry ', nan, 'Provisions, Dairy and Prepared Produce ', 'Soft Drinks/BWS ', 'Produce ', 'Pharmacy ']
tamaño de categorias: 7


In [39]:
print(data.cooling_range.unique().tolist())
print("tamaño de categorias:",len(data.cooling_range.unique()))

['-18C - less', '3C - less', nan, '5C - less', '(5-12)C - less', '7C - less', '(2-8)C - less']
tamaño de categorias: 7


Ahora podemos eliminar la columna "client_product_name"    

In [40]:
data.drop("client_product_name",axis=1,inplace=True)

### **asset_type | asset_status**    

Aparentemente requested_by, wo_priority y asset_type comparten valores vacios desde el indice (527644)

asset_type contiene una categoria llamada: "No Asset Type" lo que podria ser el mismo caso de los NaN presente en esta categoria

In [41]:
data[data.requested_by.isnull()].head(3)

,cooling_range,preserves,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,...,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,asset_type,asset_status
527644,NaN,NaN,12917191,D0470,DEPOT,2012-01-02 05:59:00,2012-03-02 10:47:00,CMS Self Delivery,EOL,REFRIGERATION,...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Depot,DA1 5PR,33424.599678,No Asset,NaN,NaN
527645,NaN,NaN,12920048,A2154,SUPERMARKET,2012-01-02 06:02:00,2012-03-13 13:14:00,Montgomery Refrigeration Ltd,EOL,REFRIGERATION,...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Region N02,EH11 2QW,49471.000000,No Asset,NaN,NaN
527646,NaN,NaN,12920042,A2031,SUPERMARKET,2012-01-02 06:02:00,2012-05-03 14:26:00,Montgomery Refrigeration Ltd,EOL,REFRIGERATION,...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Region N03,G51 4BP,46224.000000,No Asset,NaN,NaN


In [42]:
data[~data.requested_by.isnull()]

,cooling_range,preserves,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,...,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,asset_type,asset_status
0,-18C - less,Frozen Food,12904684,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 10:00:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.000000,SSL00567391,HALF GLASS DOOR & WELL,In Service
1,-18C - less,Frozen Food,12904685,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 09:59:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.000000,SSL00567398,FULL GLASS DOOR,In Service
2,-18C - less,Frozen Food,12904853,A0892,SUPERMARKET,2012-01-02 05:10:00,2012-01-02 10:01:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.000000,SSL00567399,FULL GLASS DOOR,In Service
3,3C - less,"Meat, Fish and Poultry",12927899,A0416,SUPERMARKET,2012-01-02 08:40:00,2012-02-02 13:03:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,LINDE,2002-01-08 05:00:00,Region S01,BR4 0PU,13423.000000,SSL00225160,REMOTE MULTIDECK,In Service
4,NaN,NaN,12932231,A0003,SUPERMARKET,2012-01-02 13:33:00,2012-08-02 19:39:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.000000,SSL00583503,FULL GLASS DOOR,Scrapped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527639,5C - less,"Provisions, Dairy and Prepared Produce",32704393,A0756,SUPERMARKET,2015-01-11 20:35:00,2015-01-11 20:08:00,Carter Refrigeration,Pend.,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,NUTTALL,2011-09-16 06:29:00,Region C02,ST16 2TF,41385.000000,SSL00827276,SERVE OVER,In Service
527640,-18C - less,Frozen Food,32704394,A0885,SUPERMARKET,2015-01-11 20:35:00,2015-01-11 20:08:00,KB (Carters),Assigned,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,WILLIAMS REFRIGERATION,2010-10-28 00:00:00,Region N03,G53 7RH,35575.000000,SSL00769586,INTEGRAL FREEZER,In Service
527641,-18C - less,Frozen Food,32704402,A4261,CONVENIENCE STORE,2015-01-11 20:38:00,2015-01-11 20:08:00,Space Engineering services LTD,Assigned,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,VERCO,2012-11-21 05:00:00,Region S09,PO4 0JT,33424.599678,SSL00495507,FULL GLASS DOOR,In Service
527642,3C - less,"Meat, Fish and Poultry",32704447,A4248,CONVENIENCE STORE,2015-01-11 20:50:00,2015-01-11 20:08:00,CMS Solutions Limited,Assigned,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,EPTA,2012-11-05 05:00:00,Region S04-2,E8 4HR,33424.599678,SSL00818570,REMOTE MULTIDECK,In Service


In [43]:
len(data[data.requested_by.isnull()])

82345

In [44]:
data.requested_by.value_counts()

Urgent Refrigeration Alarm                                                                        419344
Valve Open >50% for 3hrs sustained for 3 times within 24hrs (All Cabinets and Coldrooms) LIVE      42841
Air Temp Diff is Negative for a sustained 30mins within 24hr period (Cabinets) LIVE                 9444
Air Temp Diff is Negative for a sustained 40mins within 24hr period (Cabinets) LIVE                 5021
Superheat >11°C for 2hrs sustained within 24hrs LIVE                                                5020
                                                                                                   ...  
 AirOn/Off Temp Diff >8°C for 8hr within 24hr period(ProvCab) LIVE                                     1
AEL Engineer                                                                                           1
Urgent Refrigeration Alarm/K Desmond                                                                   1
AirOn/Off Temp Diff >8°C for 8hr within 24hr period(Pro

In [45]:
data[data.wo_priority == "PPM"].head(3)

,cooling_range,preserves,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,...,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,asset_type,asset_status
527644,NaN,NaN,12917191,D0470,DEPOT,2012-01-02 05:59:00,2012-03-02 10:47:00,CMS Self Delivery,EOL,REFRIGERATION,...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Depot,DA1 5PR,33424.599678,No Asset,NaN,NaN
527645,NaN,NaN,12920048,A2154,SUPERMARKET,2012-01-02 06:02:00,2012-03-13 13:14:00,Montgomery Refrigeration Ltd,EOL,REFRIGERATION,...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Region N02,EH11 2QW,49471.000000,No Asset,NaN,NaN
527646,NaN,NaN,12920042,A2031,SUPERMARKET,2012-01-02 06:02:00,2012-05-03 14:26:00,Montgomery Refrigeration Ltd,EOL,REFRIGERATION,...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Region N03,G51 4BP,46224.000000,No Asset,NaN,NaN


In [46]:
len(data[data.wo_priority == "PPM"])

82345

In [47]:
data[data.asset_type.isnull()].head(3)

,cooling_range,preserves,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,...,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,asset_type,asset_status
527644,NaN,NaN,12917191,D0470,DEPOT,2012-01-02 05:59:00,2012-03-02 10:47:00,CMS Self Delivery,EOL,REFRIGERATION,...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Depot,DA1 5PR,33424.599678,No Asset,NaN,NaN
527645,NaN,NaN,12920048,A2154,SUPERMARKET,2012-01-02 06:02:00,2012-03-13 13:14:00,Montgomery Refrigeration Ltd,EOL,REFRIGERATION,...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Region N02,EH11 2QW,49471.000000,No Asset,NaN,NaN
527646,NaN,NaN,12920042,A2031,SUPERMARKET,2012-01-02 06:02:00,2012-05-03 14:26:00,Montgomery Refrigeration Ltd,EOL,REFRIGERATION,...,NaN,PPM,No Manufacturer,2011-10-20 05:00:00,Region N03,G51 4BP,46224.000000,No Asset,NaN,NaN


In [48]:
len(data[data.asset_type.isnull()])

82345

In [49]:
data2 = data[:527644]

In [58]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527644 entries, 0 to 527643
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   cooling_range              467909 non-null  object        
 1   preserves                  467909 non-null  object        
 2   work_order_number          527644 non-null  int64         
 3   site_number                527644 non-null  object        
 4   site_type                  527644 non-null  object        
 5   date_created               527644 non-null  datetime64[ns]
 6   date_completed             527644 non-null  datetime64[ns]
 7   provider_name              527644 non-null  object        
 8   wo_current_status          527644 non-null  object        
 9   asset_category             527644 non-null  object        
 10  problem_type_name          527644 non-null  object        
 11  requested_by               527644 non-null  object  

In [62]:
data2.preserves.isnull().value_counts()

False    467909
True      59735
Name: preserves, dtype: int64

In [70]:
data2[(data2.preserves.isnull())]["site_type"].value_counts()

SUPERMARKET          47359
CONVENIENCE STORE     6489
SHARED SITE           5233
PFS                    654
Name: site_type, dtype: int64

In [88]:
def condition1(x):
    if x.isnull() and data2.site_type == "SUPERMARKET":
        return "Frozen Food"
    elif x.isnull() and data2.site_type == "CONVENIENCE STORE":
        return "Provisions, Dairy and Prepared Produce"
    elif x.isnull() and data2.site_type == "SHARED SITE":
        return "Frozen Food"
    else:
        return "Provisions, Dairy and Prepared Produce"

In [95]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [98]:
data_tc = data2[data2.preserves.isnull()]

In [107]:
data_tc.site_type.tolist()[0]

'SUPERMARKET'

In [108]:
pres_completed = []
for i in range(len(data_tc)):
    if data_tc.site_type.tolist()[i] == "SUPERMARKET":
        pres_completed.append("Frozen Food")
    elif data_tc.site_type.tolist()[i] == "CONVENIENCE STORE":
        pres_completed.append("Provisions, Dairy and Prepared Produce")
    elif data_tc.site_type.tolist()[i] == "SHARED SITE":
        pres_completed.append("Frozen Food")
    else:
        pres_completed.append("Provisions, Dairy and Prepared Produce")
    printProgressBar(i,len(data_tc)-1)

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


In [109]:
data_tc["preserves"] = pres_completed

In [ ]:
data_tc = data_tc[["preserves","work_order_number"]]
data_tc

In [112]:
C = data2.merge(data_tc, on="work_order_number", suffixes=('', '_y'), how='left')

In [114]:
C["preserves"] = C["preserves"].fillna(C['preserves_y'])
C.head()

,cooling_range,preserves,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,...,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,asset_type,asset_status,preserves_y
0,-18C - less,Frozen Food,12904684,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 10:00:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567391,HALF GLASS DOOR & WELL,In Service,NaN
1,-18C - less,Frozen Food,12904685,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 09:59:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567398,FULL GLASS DOOR,In Service,NaN
2,-18C - less,Frozen Food,12904853,A0892,SUPERMARKET,2012-01-02 05:10:00,2012-01-02 10:01:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567399,FULL GLASS DOOR,In Service,NaN
3,3C - less,"Meat, Fish and Poultry",12927899,A0416,SUPERMARKET,2012-01-02 08:40:00,2012-02-02 13:03:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,LINDE,2002-01-08 05:00:00,Region S01,BR4 0PU,13423.0,SSL00225160,REMOTE MULTIDECK,In Service,NaN
4,NaN,Frozen Food,12932231,A0003,SUPERMARKET,2012-01-02 13:33:00,2012-08-02 19:39:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00583503,FULL GLASS DOOR,Scrapped,Frozen Food


In [115]:
C.drop("cooling_range",axis=1,inplace=True)
C.drop("preserves_y",axis=1,inplace=True)

In [118]:
C.to_csv("./data/datacll.csv",index=None)

In [ ]:
data2["preserves"] = pres_completed

In [74]:
data2.asset_category.unique().tolist()

['REFRIGERATION']

In [79]:
data2[~(data2.preserves.isnull()) & (data2.site_type == "PFS")]["preserves"].value_counts()

Provisions, Dairy and Prepared Produce     4108
Soft Drinks/BWS                            1128
Meat, Fish and Poultry                      187
Frozen Food                                  38
Name: preserves, dtype: int64

In [57]:
data2.asset_status.fillna("Not Found",inplace=True)

In [59]:
data2.asset_status.unique().tolist()

['In Service',
 'Scrapped',
 'Inactive',
 'Not Verified',
 'Storage - Operable',
 'Planned Refit Removal',
 'Not Found',
 'Not Applicable',
 'Storage - Hold',
 'Storage - Needs Review',
 'Sold']

In [53]:
prueba = data2[["cooling_range","preserves"]]
prueba_nulls = prueba[prueba.preserves.isnull()]
for i in range(len(prueba_nulls)):
    if prueba_nulls.cooling_range.tolist()[i] == np.nan:
        if prueba_nulls.preserves.tolist()[i] != np.nan:
            print("true")
    else:
        if prueba_nulls.preserves.tolist()[i] == np.nan:
            print("true")

In [54]:
len(data2[data2.preserves.isnull()])

59735

In [55]:
data2[data2.preserves.isnull()]

,cooling_range,preserves,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,...,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,asset_type,asset_status
4,NaN,NaN,12932231,A0003,SUPERMARKET,2012-01-02 13:33:00,2012-08-02 19:39:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.000000,SSL00583503,FULL GLASS DOOR,Scrapped
5,NaN,NaN,12932240,A0003,SUPERMARKET,2012-01-02 13:33:00,2012-01-02 16:32:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.000000,SSL00583502,FULL GLASS DOOR,Inactive
7,NaN,NaN,12938545,A0052,SUPERMARKET,2012-02-02 08:55:00,2012-02-02 21:13:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,LINDE AG,2012-05-23 05:00:00,Region S01,CR0 4XT,48409.000000,SSL00319491,REMOTE MULTIDECK,Inactive
8,NaN,NaN,12939000,A0052,SUPERMARKET,2012-02-02 09:45:00,2012-02-02 21:16:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,LINDE AG,2012-05-23 05:00:00,Region S01,CR0 4XT,48409.000000,SSL00319502,REMOTE MULTIDECK,Inactive
9,NaN,NaN,12939001,A0052,SUPERMARKET,2012-02-02 09:45:00,2012-02-02 21:10:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,LINDE AG,2012-05-23 05:00:00,Region S01,CR0 4XT,48409.000000,SSL00319496,REMOTE MULTIDECK,Inactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525311,NaN,NaN,32578755,A0600,SUPERMARKET,2015-10-26 15:00:00,2015-10-26 19:10:00,Space Engineering services LTD,Service Complete,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,RYAN-JAYBERG LIMITED,2011-10-19 05:00:00,Region C04,OX4 4XR,44689.000000,SSL00801384,COLDROOM CONTROL PANEL,In Service
525441,NaN,NaN,32584342,A2098,SUPERMARKET,2015-10-26 22:09:00,2015-10-27 12:22:00,Montgomery Refrigeration Ltd,Pend Site Review,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,HOSHIZAKI,2010-11-03 03:00:00,Region N01,BT38 8BE,33424.599678,SSL00617597,ICE MACHINE,Scrapped
525877,NaN,NaN,32606859,A2097,SUPERMARKET,2015-10-28 09:15:00,2015-10-28 23:56:00,CBES,Pend Site Review,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,GREEN&COOL,2010-06-28 00:00:00,Region N07,BL5 3RD,33424.599678,SSL00533595,REFRIGERATION PACK,In Service
526184,NaN,NaN,32621655,A0550,SUPERMARKET,2015-10-29 02:51:00,2015-10-29 14:23:00,CBES,Service Complete,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,SPG LTD,2009-05-15 05:00:00,Region N05,CA13 9QF,15394.000000,SSL00018312,INTEGRAL MULTIDECK,In Service


In [82]:
82345/527644*100

15.606166278778874

Se observa que en "asset_status" todos los valores que eran de "client_product_name" son nulos, ademas esta columna es a su vez categorica con 12 elementos, se cambiara los NaN por "No information".

En cambio hay 86 elementos dentro de "asset_type" donde todos los valores que eran de "client_product_name" son nulos y tambien los de "asset_status". Se cambiara los NaN por "No information".

In [51]:
print( "Numero de valores en 'asset_status' con nan:",data[data.asset_status.isnull()]["work_order_number"].count() )
print( "Numero de valores en 'asset_type' con nan:",data[data.asset_type.isnull()]["work_order_number"].count() )

Numero de valores en 'asset_status' con nan: 83984
Numero de valores en 'asset_type' con nan: 82345


In [52]:
print( "Porcentaje de datos perdidos respecto al tamaño del Dataset: "+str(round((82345/609989)*100 ,2))+"%" )

Porcentaje de datos perdidos respecto al tamaño del Dataset: 13.5%


Aunque parece ser que elementos con "asset_type" nulos contiene mas nulos respecto a todo el dataset, parece adecuado eliminar esas filas pero se pierde aproximadamente el 13.5% de los datos totales que puede dañar el modelo.

In [53]:
data.asset_status.fillna(value="No information",inplace=True)
data.asset_type.fillna(value="No information",inplace=True)

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609989 entries, 0 to 609988
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   cooling_range              609989 non-null  object        
 1   preserves                  609989 non-null  object        
 2   work_order_number          609989 non-null  int64         
 3   site_number                609989 non-null  object        
 4   site_type                  609989 non-null  object        
 5   date_created               609989 non-null  datetime64[ns]
 6   date_completed             609989 non-null  datetime64[ns]
 7   provider_name              609989 non-null  object        
 8   wo_current_status          609989 non-null  object        
 9   asset_category             609989 non-null  object        
 10  problem_type_name          609989 non-null  object        
 11  requested_by               609989 non-null  object  

### Type de columnas

In [55]:
data.columns

Index(['cooling_range', 'preserves', 'work_order_number', 'site_number',
       'site_type', 'date_created', 'date_completed', 'provider_name',
       'wo_current_status', 'asset_category', 'problem_type_name',
       'requested_by', 'wo_priority', 'manufacturer', 'asset_in_service_date',
       'level_1_organization_name', 'site_zipcode', 'sales_sq_ft',
       'asset_barcode', 'asset_type', 'asset_status'],
      dtype='object')

In [56]:
for i in data.columns:
    print(data[i][0],"  ----  ",type(data[i][0]))

-18C - less   ----   <class 'str'>
Frozen Food    ----   <class 'str'>
12904684   ----   <class 'numpy.int64'>
A0892   ----   <class 'str'>
SUPERMARKET   ----   <class 'str'>
2012-01-02 03:05:00   ----   <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2012-01-02 10:00:00   ----   <class 'pandas._libs.tslibs.timestamps.Timestamp'>
Space Engineering services LTD   ----   <class 'str'>
EOL   ----   <class 'str'>
REFRIGERATION   ----   <class 'str'>
Fixture Critical Over Temperature Alarm   ----   <class 'str'>
Urgent Refrigeration Alarm   ----   <class 'str'>
Standard SLA   ----   <class 'str'>
CARTER   ----   <class 'str'>
2011-01-06 15:19:00   ----   <class 'pandas._libs.tslibs.timestamps.Timestamp'>
Region N10   ----   <class 'str'>
DN31 1UF   ----   <class 'str'>
42639.0   ----   <class 'numpy.float64'>
SSL00567391   ----   <class 'str'>
HALF GLASS DOOR & WELL   ----   <class 'str'>
In Service   ----   <class 'str'>


In [57]:
data.to_csv("./data/datacl.csv",index=None)